In [9]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [10]:
from poliprompt import TextClassifier

In [11]:
data_file = "../examples/topic_data.csv"
work_station = "../examples/TopicExperiment"
env_path = "../.env"
options = ["politics", "business", "sport", "technology", "entertainment"]
feature_col = "text"
answer_col = "label"
random_state = 42
requests_per_period = 60
seconds_per_period = 60

In [20]:
topic_classifier = TextClassifier(data_file, work_station, env_path, options)

In [18]:
embedding_llm_name = "openai"
reduce_method = "umap"
select_method = "kmeans"
testing = True
testing_size = 500
kshots = 5
lambda_param = 1.0

In [14]:
topic_classifier.create_few_shot_pool(embedding_llm_name, reduce_method, select_method, testing, testing_size)

2024-10-15 23:51:42,197 - poliprompt.text_classifier - WARNING - You are supposed to provide 'embedding_llm_configs.json', 'reduce_configs.json', and 'select_configs.json' in the path ../examples/TopicExperiment/infiles/configs
2024-10-15 23:53:26,222 - poliprompt.text_classifier - INFO - NumPy array saved to ../examples/TopicExperiment/outfiles/embeddings.npy
2024-10-15 23:53:26,241 - poliprompt.text_classifier - INFO - List of integer indices saved to ../examples/TopicExperiment/outfiles/exemplar_indices.json
2024-10-15 23:53:26,242 - poliprompt.text_classifier - INFO - Computation time: 00:01:44
2024-10-15 23:53:26,243 - poliprompt.text_classifier - INFO - You are supposed to provid label in ../examples/topic_data.csv for the next steps.


In [15]:
llm_name = "gpt-4-turbo"
prompt_file_name = "prompt.txt"

In [21]:
enhanced_description = topic_classifier.optimize_task_description(llm_name, prompt_file_name)

120it [00:00, 87716.36it/s]
/Users/jokersmith/Documents/Papers/PA24/dev/PoliPrompt/src/poliprompt/text_classifier.py:223: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  map_chain = LLMChain(llm=model, prompt=map_prompt)
/Users/jokersmith/.myenv/lib/python3.10/site-packages/langchain_openai/chat_models/base.py:360: UserWarning: Unexpected type for token usage: <class 'NoneType'>
  warnings.warn(f"Unexpected type for token usage: {type(new_usage)}")
2024-10-16 00:03:11,765 - poliprompt.text_classifier - WARNING - **Concise Summary Rules for Choosing the Correct News Report Topic:**

1. **Politics:** Choose if the text primarily discusses government actions, political figures, elections, or legislative issues.
2. **Business:** Select if the focus is on financial markets, corporate affairs, or economic policies.
3. **Sport:** Opt for this category if the text centers on s

In [22]:
enhanced_description

'**Concise Summary Rules for Choosing the Correct News Report Topic:**\n\n1. **Politics:** Choose if the text primarily discusses government actions, political figures, elections, or legislative issues.\n2. **Business:** Select if the focus is on financial markets, corporate affairs, or economic policies.\n3. **Sport:** Opt for this category if the text centers on sports events, athletes, or sports teams.\n4. **Technology:** Choose if the discussion is about technological advancements, digital trends, or tech companies.\n5. **Entertainment:** Select if the text revolves around movies, music, television, celebrities, or cultural events.\n\n**General Rule:** Always choose the category that best aligns with the central theme or primary subject matter of the text. Avoid options that do not prominently represent the content of the text.'

In [23]:
llm1_name = "gpt-3.5-turbo"
enhanced_prompt_file_name = "enhanced_prompt.txt"

In [24]:
topic_classifier.annotate(llm1_name, enhanced_prompt_file_name, kshots, lambda_param, False, testing, testing_size)

2024-10-16 00:04:59,343 - poliprompt.text_classifier - INFO - Saving output to an existing file 
../examples/TopicExperiment/outfiles/topic_data_gpt-3.5-turbo_enhanced_prompt_5shots_test.csv


Examples of data
                                                 text     label
0  Budget to set scene for election\n \n Gordon B...  politics
1  Army chiefs in regiments decision\n \n Militar...  politics
2  Howard denies split over ID cards\n \n Michael...  politics
3  Observers to monitor UK election\n \n Minister...  politics
4  Kilroy names election seat target\n \n Ex-chat...  politics


2024-10-16 00:12:16,238 - poliprompt.text_classifier - INFO - Computation time: 00:07:16
2024-10-16 00:12:16,265 - poliprompt.text_classifier - INFO - Results saved to ../examples/TopicExperiment/outfiles/topic_data_gpt-3.5-turbo_enhanced_prompt_5shots_test.csv


In [25]:
cot_llm_name = "gpt-4-turbo"
mismatch_file_name = "mismatch.json"
cot_prompt_file_name = "cot_prompt.txt"

In [26]:
topic_classifier.cot_mismatch_solver(cot_llm_name, mismatch_file_name, cot_prompt_file_name, kshots, lambda_param, False, testing, testing_size)

2024-10-16 00:13:18,944 - poliprompt.text_classifier - INFO - Computation time: 00:00:41
2024-10-16 00:13:18,956 - poliprompt.text_classifier - INFO - Results saved to ../examples/TopicExperiment/outfiles/cot_mismatch_topic_data_gpt-4-turbo_cot_prompt_5shots_test.csv


In [27]:
judge_llm_name = "gpt-4-turbo"
llm2_name = "claude-3-haiku-20240307"
cot_prompt_file_name = "cot_prompt.txt"
judge_prompt_file_name = "judge_prompt.txt"

In [28]:
topic_classifier.judge_mismatch_solver(judge_llm_name, llm1_name, llm2_name, mismatch_file_name, cot_prompt_file_name, judge_prompt_file_name, kshots, lambda_param, False, testing, testing_size)

2024-10-16 00:16:20,683 - poliprompt.text_classifier - INFO - Computation time: 00:00:54
2024-10-16 00:16:20,696 - poliprompt.text_classifier - INFO - Results saved to ../examples/TopicExperiment/outfiles/judge_mismatch_topic_data_gpt-4-turbo_judge_prompt_5shots_test.csv
